In [1]:
# ----------------------------------
# Implementation
# ----------------------------------

using Distributed
nprocs() == 1 && addprocs(; exeflags = "--project")
workers()

@everywhere using AutoMLPipeline
@everywhere using DataFrames
@everywhere using AutoMLPipeline: score
@everywhere using Random
ENV["COLUMNS"]=1000
ENV["LINES"]=100
# disable truncation of dataframes columns
import Base.show
show(df::AbstractDataFrame) = show(df,truncate=0)
show(io::IO,df::AbstractDataFrame) = show(io,df;truncate=0)

show (generic function with 361 methods)

In [2]:
# brief overview of AutoMLPipeline package
# more details is in my Juliacon2020 presentation about this package
# we use AutoMLPipeline to develop the pipeline optimizer

# define scalers
rb     = SKPreprocessor("RobustScaler",Dict(:name=>"rb"))
pt     = SKPreprocessor("PowerTransformer",Dict(:name=>"pt"))
norm   = SKPreprocessor("Normalizer",Dict(:name=>"norm"))
mx     = SKPreprocessor("MinMaxScaler",Dict(:name=>"mx"))
std    = SKPreprocessor("StandardScaler",Dict(:name=>"std"))
# define extractors
pca    = SKPreprocessor("PCA",Dict(:name=>"pca"))
fa     = SKPreprocessor("FactorAnalysis",Dict(:name=>"fa"))
ica    = SKPreprocessor("FastICA",Dict(:name=>"ica"))
# define learners
rf     = SKLearner("RandomForestClassifier",Dict(:name => "rf"))
ada    = SKLearner("AdaBoostClassifier",Dict(:name => "ada"))
gb     = SKLearner("GradientBoostingClassifier",Dict(:name => "gb"))
lsvc   = SKLearner("LinearSVC",Dict(:name => "lsvc"))
rbfsvc = SKLearner("SVC",Dict(:name => "rbfsvc"))
dt     = SKLearner("DecisionTreeClassifier",Dict(:name =>"dt"))
# preprocessing
noop = Identity(Dict(:name =>"noop"))
ohe  = OneHotEncoder(Dict(:name=>"ohe"))
catf = CatFeatureSelector(Dict(:name=>"catf"))
numf = NumFeatureSelector(Dict(:name=>"numf"))

vscalers =    [rb,pt,norm,mx,std,noop]
vextractors = [pca,fa,ica,noop]
vlearners =   [rf,gb,lsvc,rbfsvc,ada,dt]

dataset = getiris()
X = dataset[:,1:4]
Y = dataset[:,5] |> collect;

In [3]:
# one-block pipeline
prep = @pipeline (catf |> ohe) + numf
ppl1 = @pipeline prep |> (mx |> pca) |> rf
crossvalidate(ppl1,X,Y)

fold: 1, 93.33333333333333
fold: 2, 93.33333333333333
fold: 3, 80.0
fold: 4, 100.0
fold: 5, 100.0
fold: 6, 100.0
fold: 7, 93.33333333333333
fold: 8, 100.0
fold: 9, 93.33333333333333
fold: 10, 100.0
errors: 0


┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /Users/ppalmes/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


(mean = 95.33333333333334, std = 6.32455532033676, folds = 10, errors = 0)

In [4]:
# two-block pipeline
ppl2 = @pipeline prep |> ((norm |> ica) + (pt |> pca)) |> lsvc
crossvalidate(ppl2,X,Y)

fold: 1, 93.33333333333333
fold: 2, 93.33333333333333
fold: 3, 93.33333333333333
fold: 4, 100.0
fold: 5, 93.33333333333333
fold: 6, 100.0
fold: 7, 100.0
fold: 8, 100.0
fold: 9, 100.0
fold: 10, 93.33333333333333
errors: 0


(mean = 96.66666666666666, std = 3.5136418446315347, folds = 10, errors = 0)

In [5]:
function oneblock_pipeline_factory(scalers,extractors,learners)
   results = @distributed (vcat) for lr in learners
      @distributed (vcat) for xt in extractors
         @distributed (vcat) for sc in scalers
            # baseline preprocessing
            prep = @pipeline ((catf |> ohe) + numf)
            # one-block prp
            expx = @pipeline prep |> (sc |> xt) |> lr
            scn   = sc.name[1:end - 4];xtn = xt.name[1:end - 4]; lrn = lr.name[1:end - 4]
            pname = "($scn |> $xtn) |> $lrn"
            DataFrame(Description=pname,Pipeline=expx)
         end
      end
   end
   return results
end

oneblock_pipeline_factory (generic function with 1 method)

In [6]:
res_oneblock = oneblock_pipeline_factory(vscalers,vextractors,vlearners)

,Description,Pipeline
,String,Pipeline
1,(rb |> pca) |> rf,"Pipeline(""linearpipeline_sfQ"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_sfQ"", :machines=>Machine[Pipeline(""linearpipeline_lfQ"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_lfQ"", :machines=>Workflow[ComboPipeline(""combopipeline_Z5u"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_Z5u"", :machines=>Machine[Pipeline(""linearpipeline_9gg"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_9gg"", :machines=>Transformer[CatFeatureSelector(""catf_quI"", Dict{Symbol, Any}(:name=>""catf_quI"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_sK1"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_sK1"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_aSP"", Dict{Symbol, Any}(:name=>""numf_aSP"", :numcols=>Int64[]))])), Pipeline(""linearpipeline_jD8"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_jD8"", :machines=>SKPreprocessor[SKPreprocessor(""rb_kgQ"", Dict{Symbol, Any}(:name=>""rb_kgQ"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_ifc"", Dict{Symbol, Any}(:name=>""pca_ifc"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))])), SKLearner(""rf_0pd"", Dict{Symbol, Any}(:output=>:class, :name=>""rf_0pd"", :impl_args=>Dict{Symbol, Any}(), :learner=>""RandomForestClassifier""))]))"
2,(pt |> pca) |> rf,"Pipeline(""linearpipeline_NoN"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_NoN"", :machines=>Machine[Pipeline(""linearpipeline_cUW"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_cUW"", :machines=>Workflow[ComboPipeline(""combopipeline_rTc"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_rTc"", :machines=>Machine[Pipeline(""linearpipeline_heS"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_heS"", :machines=>Transformer[CatFeatureSelector(""catf_quI"", Dict{Symbol, Any}(:name=>""catf_quI"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_sK1"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_sK1"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_aSP"", Dict{Symbol, Any}(:name=>""numf_aSP"", :numcols=>Int64[]))])), Pipeline(""linearpipeline_Ydo"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_Ydo"", :machines=>SKPreprocessor[SKPreprocessor(""pt_zwI"", Dict{Symbol, Any}(:name=>""pt_zwI"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PowerTransformer"")), SKPreprocessor(""pca_ifc"", Dict{Symbol, Any}(:name=>""pca_ifc"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))])), SKLearner(""rf_0pd"", Dict{Symbol, Any}(:output=>:class, :name=>""rf_0pd"", :impl_args=>Dict{Symbol, Any}(), :learner=>""RandomForestClassifier""))]))"
3,(norm |> pca) |> rf,"Pipeline(""linearpipeline_8xP"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_8xP"", :machines=>Machine[Pipeline(""linearpipeline_jMC"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_jMC"", :machines=>Workflow[ComboPipeline(""combopipeline_qyl"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_qyl"", :machines=>Machine[Pipeline(""linearpipeline_stF"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_stF"", :machines=>Transformer[CatFeatureSelector(""catf_quI"", Dict{Symbol, Any}(:name=>""catf_quI"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_sK1"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_sK1"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_aSP"", Dict{Symbol, Any}(:name=>""numf_aSP"", :numcols=>Int64[]))])), 

In [7]:
function evaluate_pipeline(dfpipelines,X,Y;folds=3)
   res=@distributed (vcat) for prow in eachrow(dfpipelines)
      perf = crossvalidate(prow.Pipeline,X,Y;nfolds=folds)
      DataFrame(Description=prow.Description,mean=perf.mean,sd=perf.std)
   end
   return res
end

evaluate_pipeline (generic function with 1 method)

In [8]:
cross_res_oneblock = evaluate_pipeline(res_oneblock,X,Y)

      From worker 3:	┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
      From worker 3:	│   caller = npyinitialize() at numpy.jl:67
      From worker 3:	└ @ PyCall ~/.julia/packages/PyCall/L0fLP/src/numpy.jl:67
      From worker 6:	┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
      From worker 6:	│   caller = npyinitialize() at numpy.jl:67
      From worker 6:	└ @ PyCall ~/.julia/packages/PyCall/L0fLP/src/numpy.jl:67
      From worker 5:	┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
      From worker 5:	│   caller = npyinitialize() at numpy.jl:67
      From worker 5:	└ @ PyCall ~/.julia/packages/PyCall/L0fLP/src/numpy.jl:67
      From worker 7:	┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
      From worker 7:	│   caller = npyinitialize() at numpy.jl:67
      From worker 7:	└ @ PyCall ~/.julia/packages/P

      From worker 7:	fold: 3, 94.0
      From worker 7:	errors: 0
      From worker 5:	fold: 2, 90.0
      From worker 5:	fold: 3, 96.0
      From worker 5:	errors: 0
      From worker 5:	fold: 1, 92.0
      From worker 5:	fold: 2, 98.0
      From worker 5:	fold: 3, 94.0
      From worker 5:	errors: 0
      From worker 8:	┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
      From worker 8:	│   caller = npyinitialize() at numpy.jl:67
      From worker 8:	└ @ PyCall ~/.julia/packages/PyCall/L0fLP/src/numpy.jl:67
      From worker 5:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
      From worker 5:	  warnings.warn(
      From worker 5:	fold: 1, 96.0
      From worker 5:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
 

      From worker 9:	errors: 0
      From worker 9:	fold: 1, 100.0
      From worker 9:	fold: 2, 92.0
      From worker 9:	fold: 3, 94.0
      From worker 9:	errors: 0
      From worker 2:	fold: 2, 92.0
      From worker 3:	fold: 2, 98.0
      From worker 4:	fold: 2, 92.0
      From worker 8:	fold: 1, 86.0
      From worker 2:	fold: 3, 86.0
      From worker 2:	errors: 0
      From worker 8:	fold: 2, 66.0
      From worker 3:	fold: 3, 100.0
      From worker 3:	errors: 0
      From worker 8:	fold: 3, 92.0
      From worker 8:	errors: 0
      From worker 4:	fold: 3, 92.0
      From worker 4:	errors: 0
      From worker 2:	fold: 1, 90.0
      From worker 8:	fold: 1, 70.0
      From worker 8:	fold: 2, 94.0
      From worker 3:	fold: 1, 94.0
      From worker 2:	fold: 2, 94.0
      From worker 8:	fold: 3, 96.0
      From worker 8:	errors: 0
      From worker 4:	fold: 1, 96.0
      From worker 8:	fold: 1, 86.0
      From worker 2:	fold: 3, 90.0
      From worker 2:	errors: 0
      From work

,Description,mean,sd
,String,Float64,Float64
1,(rb |> pca) |> rf,92.6667,5.03322
2,(pt |> pca) |> rf,93.3333,5.7735
3,(norm |> pca) |> rf,98.0,2.0
4,(mx |> pca) |> rf,95.3333,3.05505
5,(std |> pca) |> rf,96.0,5.2915
6,(noop |> pca) |> rf,95.3333,1.1547
7,(rb |> fa) |> rf,91.3333,5.03322
8,(pt |> fa) |> rf,91.3333,2.3094
9,(norm |> fa) |> rf,93.3333,3.05505


In [9]:
sort!(cross_res_oneblock,:mean, rev = true)

,Description,mean,sd
,String,Float64,Float64
1,(pt |> ica) |> rf,98.6667,2.3094
2,(norm |> pca) |> rf,98.0,2.0
3,(norm |> pca) |> gb,98.0,3.4641
4,(mx |> ica) |> dt,98.0,2.0
5,(norm |> noop) |> rf,97.3333,3.05505
6,(noop |> pca) |> rbfsvc,97.3333,1.1547
7,(norm |> pca) |> ada,97.3333,1.1547
8,(noop |> fa) |> gb,96.6667,4.16333
9,(mx |> ica) |> gb,96.6667,3.05505


In [10]:
function twoblock_pipeline_factory(scalers,extractors,learners)
   results = @distributed (vcat) for lr in learners
      @distributed (vcat) for xt1 in extractors
         @distributed (vcat) for xt2 in extractors
            @distributed (vcat) for sc1 in scalers
               @distributed (vcat) for sc2 in scalers
                  prep = @pipeline ((catf |> ohe) + numf)
                  expx = @pipeline prep |> ((sc1 |> xt1) + (sc2 |> xt2)) |> lr
                  scn1   = sc1.name[1:end - 4];xtn1 = xt1.name[1:end - 4];
                  scn2   = sc2.name[1:end - 4];xtn2 = xt2.name[1:end - 4];
                  lrn = lr.name[1:end - 4]
                  pname = "($scn1 |> $xtn1) + ($scn2 |> $xtn2) |> $lrn"
                  DataFrame(Description=pname,Pipeline=expx)
               end
            end
         end
      end
   end
   return results
end
res_twoblock = twoblock_pipeline_factory(vscalers,vextractors,vlearners)

,Description,Pipeline
,String,Pipeline
1,(rb |> pca) + (rb |> pca) |> rf,"Pipeline(""linearpipeline_CtE"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_CtE"", :machines=>Machine[Pipeline(""linearpipeline_Ket"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_Ket"", :machines=>ComboPipeline[ComboPipeline(""combopipeline_EVm"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_EVm"", :machines=>Machine[Pipeline(""linearpipeline_Bmn"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_Bmn"", :machines=>Transformer[CatFeatureSelector(""catf_quI"", Dict{Symbol, Any}(:name=>""catf_quI"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_sK1"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_sK1"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_aSP"", Dict{Symbol, Any}(:name=>""numf_aSP"", :numcols=>Int64[]))])), ComboPipeline(""combopipeline_n8H"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_n8H"", :machines=>Pipeline[Pipeline(""linearpipeline_8Ud"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_8Ud"", :machines=>SKPreprocessor[SKPreprocessor(""rb_kgQ"", Dict{Symbol, Any}(:name=>""rb_kgQ"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_ifc"", Dict{Symbol, Any}(:name=>""pca_ifc"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))])), Pipeline(""linearpipeline_C3a"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_C3a"", :machines=>SKPreprocessor[SKPreprocessor(""rb_kgQ"", Dict{Symbol, Any}(:name=>""rb_kgQ"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_ifc"", Dict{Symbol, Any}(:name=>""pca_ifc"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))]))])), SKLearner(""rf_0pd"", Dict{Symbol, Any}(:output=>:class, :name=>""rf_0pd"", :impl_args=>Dict{Symbol, Any}(), :learner=>""RandomForestClassifier""))]))"
2,(rb |> pca) + (pt |> pca) |> rf,"Pipeline(""linearpipeline_ign"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_ign"", :machines=>Machine[Pipeline(""linearpipeline_9j1"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_9j1"", :machines=>ComboPipeline[ComboPipeline(""combopipeline_Ku0"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_Ku0"", :machines=>Machine[Pipeline(""linearpipeline_JE3"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_JE3"", :machines=>Transformer[CatFeatureSelector(""catf_quI"", Dict{Symbol, Any}(:name=>""catf_quI"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_sK1"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_sK1"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_aSP"", Dict{Symbol, Any}(:name=>""numf_aSP"", :numcols=>Int64[]))])), ComboPipeline(""combopipeline_ZqG"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_ZqG"", :machines=>Pipeline[Pipeline(""linearpipeline_SWL"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_SWL"", :machines=>SKPreprocessor[SKPreprocessor(""rb_kgQ"", Dict{Symbol, Any}(:name=>""rb_kgQ"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_ifc"", Dict{Symbol, Any}(:name=>""pca_ifc"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))])), Pipeline(""linearpipeline_fHT"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_fHT"", :machines=>SKPreprocessor[SKPreprocessor(""pt_zwI"", Dict{Symbol, Any}(:name=>""pt_zwI"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PowerTransformer"")), SKPreprocesso

In [11]:
function model_selection_pipeline(learners)
   results = @distributed (vcat) for lr in learners
      prep = @pipeline ((catf |> ohe) + numf)
      expx = @pipeline prep |> (rb |> pca)  |> lr
      pname = "(rb |> pca) |> $(lr.name[1:end-4])"
      DataFrame(Description=pname,Pipeline=expx)
   end
   return results
end
dfpipes = model_selection_pipeline(vlearners)

,Description,Pipeline
,String,Pipeline
1,(rb |> pca) |> rf,"Pipeline(""linearpipeline_YF9"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_YF9"", :machines=>Machine[Pipeline(""linearpipeline_WnK"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_WnK"", :machines=>Workflow[ComboPipeline(""combopipeline_B2J"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_B2J"", :machines=>Machine[Pipeline(""linearpipeline_dFd"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_dFd"", :machines=>Transformer[CatFeatureSelector(""catf_quI"", Dict{Symbol, Any}(:name=>""catf_quI"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_sK1"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_sK1"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_aSP"", Dict{Symbol, Any}(:name=>""numf_aSP"", :numcols=>Int64[]))])), Pipeline(""linearpipeline_3gR"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_3gR"", :machines=>SKPreprocessor[SKPreprocessor(""rb_kgQ"", Dict{Symbol, Any}(:name=>""rb_kgQ"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_ifc"", Dict{Symbol, Any}(:name=>""pca_ifc"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))])), SKLearner(""rf_0pd"", Dict{Symbol, Any}(:output=>:class, :name=>""rf_0pd"", :impl_args=>Dict{Symbol, Any}(), :learner=>""RandomForestClassifier""))]))"
2,(rb |> pca) |> gb,"Pipeline(""linearpipeline_a7x"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_a7x"", :machines=>Machine[Pipeline(""linearpipeline_nPh"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_nPh"", :machines=>Workflow[ComboPipeline(""combopipeline_5HU"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_5HU"", :machines=>Machine[Pipeline(""linearpipeline_nyo"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_nyo"", :machines=>Transformer[CatFeatureSelector(""catf_quI"", Dict{Symbol, Any}(:name=>""catf_quI"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_sK1"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_sK1"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_aSP"", Dict{Symbol, Any}(:name=>""numf_aSP"", :numcols=>Int64[]))])), Pipeline(""linearpipeline_FvU"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_FvU"", :machines=>SKPreprocessor[SKPreprocessor(""rb_kgQ"", Dict{Symbol, Any}(:name=>""rb_kgQ"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_ifc"", Dict{Symbol, Any}(:name=>""pca_ifc"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))])), SKLearner(""gb_Jrn"", Dict{Symbol, Any}(:output=>:class, :name=>""gb_Jrn"", :impl_args=>Dict{Symbol, Any}(), :learner=>""GradientBoostingClassifier""))]))"
3,(rb |> pca) |> lsvc,"Pipeline(""linearpipeline_bZO"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_bZO"", :machines=>Machine[Pipeline(""linearpipeline_5oI"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_5oI"", :machines=>Workflow[ComboPipeline(""combopipeline_bIq"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_bIq"", :machines=>Machine[Pipeline(""linearpipeline_1sS"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_1sS"", :machines=>Transformer[CatFeatureSelector(""catf_quI"", Dict{Symbol, Any}(:name=>""catf_quI"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_sK1"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_sK1"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_aSP"", Dict{Symbol, Any}(:name=>""numf_aSP"", :numcols=>Int64[]))])), 

In [12]:
# find the best model
@everywhere Random.seed!(10)
res = evaluate_pipeline(dfpipes,X,Y;folds=10)
sort!(res,:mean, rev = true)

      From worker 5:	fold: 1, 100.0
      From worker 4:	fold: 1, 93.33333333333333
      From worker 7:	fold: 1, 93.33333333333333
      From worker 5:	fold: 2, 100.0
      From worker 4:	fold: 2, 100.0
      From worker 7:	fold: 2, 93.33333333333333
      From worker 5:	fold: 3, 100.0
      From worker 4:	fold: 3, 100.0
      From worker 7:	fold: 3, 86.66666666666667
      From worker 5:	fold: 4, 100.0
      From worker 4:	fold: 4, 93.33333333333333
      From worker 5:	fold: 5, 86.66666666666667
      From worker 7:	fold: 4, 93.33333333333333
      From worker 5:	fold: 6, 93.33333333333333
      From worker 4:	fold: 5, 93.33333333333333
      From worker 7:	fold: 5, 66.66666666666666
      From worker 5:	fold: 7, 93.33333333333333
      From worker 4:	fold: 6, 93.33333333333333
      From worker 7:	fold: 6, 93.33333333333333
      From worker 5:	fold: 8, 100.0
      From worker 4:	fold: 7, 93.33333333333333
      From worker 7:	fold: 7, 80.0
      From worker 5:	fold: 9, 86.66666666

,Description,mean,sd
,String,Float64,Float64
1,(rb |> pca) |> rbfsvc,95.3333,5.48848
2,(rb |> pca) |> lsvc,93.3333,4.44444
3,(rb |> pca) |> gb,92.0,9.32275
4,(rb |> pca) |> rf,91.3333,7.06233
5,(rb |> pca) |> dt,87.3333,8.57789
6,(rb |> pca) |> ada,79.3333,8.57789


In [13]:
# use the best model to generate pipeline search
dfp = twoblock_pipeline_factory(vscalers,vextractors,[rbfsvc])

,Description,Pipeline
,String,Pipeline
1,(rb |> pca) + (rb |> pca) |> rbfsvc,"Pipeline(""linearpipeline_n4G"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_n4G"", :machines=>Machine[Pipeline(""linearpipeline_Mje"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_Mje"", :machines=>ComboPipeline[ComboPipeline(""combopipeline_Ycx"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_Ycx"", :machines=>Machine[Pipeline(""linearpipeline_ocn"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_ocn"", :machines=>Transformer[CatFeatureSelector(""catf_quI"", Dict{Symbol, Any}(:name=>""catf_quI"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_sK1"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_sK1"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_aSP"", Dict{Symbol, Any}(:name=>""numf_aSP"", :numcols=>Int64[]))])), ComboPipeline(""combopipeline_hDi"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_hDi"", :machines=>Pipeline[Pipeline(""linearpipeline_agM"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_agM"", :machines=>SKPreprocessor[SKPreprocessor(""rb_kgQ"", Dict{Symbol, Any}(:name=>""rb_kgQ"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_ifc"", Dict{Symbol, Any}(:name=>""pca_ifc"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))])), Pipeline(""linearpipeline_7F3"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_7F3"", :machines=>SKPreprocessor[SKPreprocessor(""rb_kgQ"", Dict{Symbol, Any}(:name=>""rb_kgQ"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_ifc"", Dict{Symbol, Any}(:name=>""pca_ifc"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))]))])), SKLearner(""rbfsvc_qVp"", Dict{Symbol, Any}(:output=>:class, :name=>""rbfsvc_qVp"", :impl_args=>Dict{Symbol, Any}(), :learner=>""SVC""))]))"
2,(rb |> pca) + (pt |> pca) |> rbfsvc,"Pipeline(""linearpipeline_xAu"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_xAu"", :machines=>Machine[Pipeline(""linearpipeline_QDg"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_QDg"", :machines=>ComboPipeline[ComboPipeline(""combopipeline_daG"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_daG"", :machines=>Machine[Pipeline(""linearpipeline_qDX"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_qDX"", :machines=>Transformer[CatFeatureSelector(""catf_quI"", Dict{Symbol, Any}(:name=>""catf_quI"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_sK1"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_sK1"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_aSP"", Dict{Symbol, Any}(:name=>""numf_aSP"", :numcols=>Int64[]))])), ComboPipeline(""combopipeline_0RX"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_0RX"", :machines=>Pipeline[Pipeline(""linearpipeline_gWt"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_gWt"", :machines=>SKPreprocessor[SKPreprocessor(""rb_kgQ"", Dict{Symbol, Any}(:name=>""rb_kgQ"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_ifc"", Dict{Symbol, Any}(:name=>""pca_ifc"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))])), Pipeline(""linearpipeline_eKV"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_eKV"", :machines=>SKPreprocessor[SKPreprocessor(""pt_zwI"", Dict{Symbol, Any}(:name=>""pt_zwI"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PowerTransformer"")), SKPreprocessor("

In [14]:
# evaluate the pipeline
@everywhere Random.seed!(10)
bestp=evaluate_pipeline(dfp,X,Y;folds=3)

      From worker 4:	fold: 1, 94.0
      From worker 3:	fold: 1, 94.0
      From worker 5:	fold: 1, 96.0
      From worker 6:	fold: 1, 92.0
      From worker 4:	fold: 2, 96.0
      From worker 2:	fold: 1, 96.0
      From worker 8:	fold: 1, 94.0
      From worker 4:	fold: 3, 98.0
      From worker 4:	errors: 0
      From worker 2:	fold: 2, 94.0
      From worker 8:	fold: 2, 98.0
      From worker 5:	fold: 2, 98.0
      From worker 2:	fold: 3, 94.0
      From worker 2:	errors: 0
      From worker 8:	fold: 3, 96.0
      From worker 8:	errors: 0
      From worker 4:	fold: 1, 98.0
      From worker 8:	fold: 1, 92.0
      From worker 2:	fold: 1, 94.0
      From worker 6:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
      From worker 6:	  warnings.warn(
      From worker 3:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decompo

      From worker 7:	  warnings.warn(
      From worker 7:	fold: 3, 94.0
      From worker 8:	fold: 3, 100.0
      From worker 2:	fold: 2, 92.0
      From worker 7:	errors: 0
      From worker 3:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
      From worker 3:	  warnings.warn(
      From worker 8:	errors: 0
      From worker 6:	fold: 1, 96.0
      From worker 5:	fold: 2, 94.0
      From worker 3:	fold: 1, 90.0
      From worker 6:	fold: 2, 98.0
      From worker 7:	fold: 1, 100.0
      From worker 8:	fold: 1, 94.0
      From worker 2:	fold: 3, 98.0
      From worker 2:	errors: 0
      From worker 3:	fold: 2, 98.0
      From worker 3:	fold: 3, 94.0
      From worker 3:	errors: 0
      From worker 9:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge.

      From worker 7:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
      From worker 7:	  warnings.warn(
      From worker 2:	fold: 3, 96.0
      From worker 2:	errors: 0
      From worker 3:	fold: 2, 90.0
      From worker 8:	fold: 3, 100.0
      From worker 8:	errors: 0
      From worker 8:	fold: 1, 98.0
      From worker 8:	fold: 2, 98.0
      From worker 6:	fold: 1, 92.0
      From worker 8:	fold: 3, 96.0
      From worker 8:	errors: 0
      From worker 9:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
      From worker 9:	  warnings.warn(
      From worker 2:	fold: 1, 94.0
      From worker 8:	fold: 1, 98.0
      From worker 9:	fold: 2, 94.0
      From worker 4:	fold: 3, 94.0

      From worker 2:	errors: 0
      From worker 4:	fold: 2, 90.0
      From worker 8:	fold: 2, 94.0
      From worker 9:	fold: 1, 94.0
      From worker 8:	fold: 3, 96.0
      From worker 8:	errors: 0
      From worker 2:	fold: 1, 96.0
      From worker 8:	fold: 1, 98.0
      From worker 8:	fold: 2, 100.0
      From worker 4:	fold: 3, 98.0
      From worker 4:	errors: 0
      From worker 8:	fold: 3, 88.0
      From worker 8:	errors: 0
      From worker 8:	fold: 1, 92.0
      From worker 5:	fold: 2, 92.0
      From worker 2:	fold: 2, 98.0
      From worker 8:	fold: 2, 90.0
      From worker 7:	fold: 1, 90.0
      From worker 9:	fold: 2, 98.0
      From worker 8:	fold: 3, 100.0
      From worker 8:	errors: 0
      From worker 4:	fold: 1, 94.0
      From worker 2:	fold: 3, 94.0
      From worker 2:	errors: 0
      From worker 3:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolera

      From worker 2:	fold: 2, 96.0
      From worker 6:	fold: 2, 96.0
      From worker 9:	fold: 1, 90.0
      From worker 3:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
      From worker 3:	  warnings.warn(
      From worker 4:	fold: 3, 98.0
      From worker 6:	fold: 3, 96.0
      From worker 4:	errors: 0
      From worker 6:	errors: 0
      From worker 2:	fold: 3, 94.0
      From worker 2:	errors: 0
      From worker 7:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
      From worker 7:	  warnings.warn(
      From worker 8:	fold: 2, 94.0
      From worker 3:	fold: 3, 92.0
      From worker 3:	errors: 0
      From worker 4:	fold: 1, 96.0
      From worker 3:	fold: 1, 98.0
    

...

      From worker 6:	  warnings.warn(
      From worker 5:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
      From worker 5:	  warnings.warn(
      From worker 9:	fold: 2, 98.0
      From worker 3:	fold: 3, 98.0
      From worker 3:	errors: 0
      From worker 5:	fold: 3, 96.0
      From worker 5:	errors: 0
      From worker 6:	fold: 3, 96.0
      From worker 6:	errors: 0
      From worker 4:	fold: 1, 92.0
      From worker 7:	fold: 2, 94.0
      From worker 5:	fold: 1, 96.0
      From worker 9:	fold: 3, 92.0
      From worker 9:	errors: 0
      From worker 3:	fold: 1, 92.0
      From worker 6:	fold: 1, 94.0
      From worker 9:	fold: 1, 96.0
      From worker 4:	fold: 2, 98.0
      From worker 9:	fold: 2, 92.0
      From worker 6:	fold: 2, 96.0
      From worker 9:	fold: 3, 96.0
      From worker 9:	errors: 0
      From worker 3:	

      From worker 3:	fold: 1, 94.0
      From worker 6:	fold: 3, 94.0
      From worker 6:	errors: 0
      From worker 3:	fold: 2, 98.0
      From worker 9:	fold: 1, 96.0
      From worker 7:	fold: 1, 92.0
      From worker 3:	fold: 3, 98.0
      From worker 3:	errors: 0
      From worker 3:	fold: 1, 98.0
      From worker 9:	fold: 2, 92.0
      From worker 3:	fold: 2, 94.0
      From worker 5:	fold: 3, 96.0
      From worker 5:	errors: 0
      From worker 3:	fold: 3, 96.0
      From worker 3:	errors: 0
      From worker 3:	fold: 1, 98.0
      From worker 9:	fold: 3, 98.0
      From worker 9:	errors: 0
      From worker 3:	fold: 2, 94.0
      From worker 9:	fold: 1, 92.0
      From worker 3:	fold: 3, 94.0
      From worker 3:	errors: 0
      From worker 5:	fold: 1, 96.0
      From worker 9:	fold: 2, 100.0
      From worker 9:	fold: 3, 96.0
      From worker 9:	errors: 0
      From worker 3:	fold: 1, 96.0
      From worker 9:	fold: 1, 92.0
      From worker 9:	fold: 2, 94.0
      From w

      From worker 6:	  warnings.warn(
      From worker 5:	fold: 2, 90.0
      From worker 6:	fold: 1, 98.0
      From worker 5:	fold: 3, 92.0
      From worker 5:	errors: 0
      From worker 5:	fold: 1, 94.0
      From worker 5:	fold: 2, 98.0
      From worker 5:	fold: 3, 94.0
      From worker 5:	errors: 0
      From worker 5:	fold: 1, 94.0
      From worker 5:	fold: 2, 96.0
      From worker 7:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
      From worker 7:	  warnings.warn(
      From worker 6:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
      From worker 6:	  warnings.warn(
      From worker 7:	fold: 3, 92.0
      From worker 7:	errors: 0
      From worker 5:	fold: 3, 98

      From worker 6:	  warnings.warn(
      From worker 6:	fold: 2, 94.0
      From worker 7:	fold: 2, 94.0
      From worker 6:	fold: 3, 98.0
      From worker 6:	errors: 0
      From worker 6:	fold: 1, 94.0
      From worker 7:	fold: 3, 92.0
      From worker 7:	errors: 0
      From worker 7:	fold: 1, 94.0
      From worker 6:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
      From worker 6:	  warnings.warn(
      From worker 6:	fold: 2, 94.0
      From worker 7:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
      From worker 7:	  warnings.warn(
      From worker 7:	fold: 2, 90.0
      From worker 6:	/Users/ppalmes/.local/lib/python3.9/site-packages/sklearn/decomposition/_fast

,Description,mean,sd
,String,Float64,Float64
1,(rb |> pca) + (rb |> pca) |> rbfsvc,94.6667,1.1547
2,(rb |> pca) + (pt |> pca) |> rbfsvc,95.3333,2.3094
3,(rb |> pca) + (norm |> pca) |> rbfsvc,93.3333,1.1547
4,(rb |> pca) + (mx |> pca) |> rbfsvc,93.3333,7.02377
5,(rb |> pca) + (std |> pca) |> rbfsvc,94.6667,4.16333
6,(rb |> pca) + (noop |> pca) |> rbfsvc,97.3333,3.05505
7,(pt |> pca) + (rb |> pca) |> rbfsvc,95.3333,1.1547
8,(pt |> pca) + (pt |> pca) |> rbfsvc,96.6667,2.3094
9,(pt |> pca) + (norm |> pca) |> rbfsvc,94.6667,3.05505


In [15]:
sort!(bestp,:mean, rev = true)

,Description,mean,sd
,String,Float64,Float64
1,(mx |> pca) + (norm |> fa) |> rbfsvc,98.6667,2.3094
2,(std |> pca) + (norm |> fa) |> rbfsvc,98.6667,1.1547
3,(noop |> pca) + (rb |> ica) |> rbfsvc,98.6667,1.1547
4,(norm |> fa) + (rb |> fa) |> rbfsvc,98.6667,2.3094
5,(pt |> ica) + (norm |> fa) |> rbfsvc,98.6667,2.3094
6,(mx |> ica) + (norm |> fa) |> rbfsvc,98.6667,1.1547
7,(rb |> ica) + (norm |> ica) |> rbfsvc,98.6667,1.1547
8,(norm |> ica) + (rb |> ica) |> rbfsvc,98.6667,1.1547
9,(norm |> noop) + (mx |> noop) |> rbfsvc,98.6667,1.1547


In [16]:
show(bestp;allrows=true,truncate=0,allcols=true)

576×3 DataFrame
 Row │ Description                                mean     sd       
     │ String                                     Float64  Float64  
─────┼──────────────────────────────────────────────────────────────
   1 │ (mx |> pca) + (norm |> fa) |> rbfsvc       98.6667   2.3094
   2 │ (std |> pca) + (norm |> fa) |> rbfsvc      98.6667   1.1547
   3 │ (noop |> pca) + (rb |> ica) |> rbfsvc      98.6667   1.1547
   4 │ (norm |> fa) + (rb |> fa) |> rbfsvc        98.6667   2.3094
   5 │ (pt |> ica) + (norm |> fa) |> rbfsvc       98.6667   2.3094
   6 │ (mx |> ica) + (norm |> fa) |> rbfsvc       98.6667   1.1547
   7 │ (rb |> ica) + (norm |> ica) |> rbfsvc      98.6667   1.1547
   8 │ (norm |> ica) + (rb |> ica) |> rbfsvc      98.6667   1.1547
   9 │ (norm |> noop) + (mx |> noop) |> rbfsvc    98.6667   1.1547
  10 │ (noop |> pca) + (noop |> pca) |> rbfsvc    98.0      2.0
  11 │ (norm |> pca) + (norm |> ica) |> rbfsvc    98.0      2.0
  12 │ (noop |> pca) + (mx |> ica) |> rbfsvc  